In [3]:
import pyspark
from pyspark import SparkContext as spark
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder.enableHiveSupport().getOrCreate()
sc = spark.sparkContext

In [4]:
df = spark.read.json('transacoes2.json')

In [16]:
df.show()

+--------------------+
|     _corrupt_record|
+--------------------+
|["{"transacoes": ...|
+--------------------+



In [21]:
import json

In [43]:
j = [
    {
       "transacao_id":1,
       "total_bruto":3000,
       "desconto_percentual":6.99
    },
    {
       "transacao_id":2,
       "total_bruto":57989,
       "desconto_percentual":1.45
    },
    {
       "transacao_id":4,
       "total_bruto":1,
       "desconto_percentual":"None"
    },
    {
       "transacao_id":5,
       "total_bruto":34,
       "desconto_percentual":0.0
    }
 ]

In [44]:
a=[json.dumps(j)]

In [45]:
jsonRDD = sc.parallelize(a)

In [46]:
df = spark.read.json(jsonRDD)

In [47]:
df.show()

+-------------------+-----------+------------+
|desconto_percentual|total_bruto|transacao_id|
+-------------------+-----------+------------+
|               6.99|       3000|           1|
|               1.45|      57989|           2|
|               None|          1|           4|
|                0.0|         34|           5|
+-------------------+-----------+------------+



In [48]:
from pyspark.sql.functions import col,when

In [49]:
df.withColumn("desconto", col("total_bruto")*(col("desconto_percentual")/100)).show()

+-------------------+-----------+------------+------------------+
|desconto_percentual|total_bruto|transacao_id|          desconto|
+-------------------+-----------+------------+------------------+
|               6.99|       3000|           1|209.70000000000002|
|               1.45|      57989|           2| 840.8404999999999|
|               None|          1|           4|              null|
|                0.0|         34|           5|               0.0|
+-------------------+-----------+------------+------------------+



In [50]:
df2 = df.withColumn("desconto", col("total_bruto")*(col("desconto_percentual")/100))

In [54]:
df3 = df2.withColumn("total_per_id", col("total_bruto") - col("desconto"))

In [55]:
df3.show()

+-------------------+-----------+------------+------------------+------------+
|desconto_percentual|total_bruto|transacao_id|          desconto|total_per_id|
+-------------------+-----------+------------+------------------+------------+
|               6.99|       3000|           1|209.70000000000002|      2790.3|
|               1.45|      57989|           2| 840.8404999999999|  57148.1595|
|               None|          1|           4|              null|        null|
|                0.0|         34|           5|               0.0|        34.0|
+-------------------+-----------+------------+------------------+------------+



In [69]:
df3.groupBy().agg(sum(col("total_per_id"))).show()

+------------------+
| sum(total_per_id)|
+------------------+
|59972.459500000004|
+------------------+

